<a href="https://colab.research.google.com/github/kate-wolfe/DEI/blob/main/DivAudit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Mount Google Drive. This can also be done by clicking on the folder on the left and choosing the "mount drive" option.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Import libraries

In [ ]:
import pandas as pd
import regex as re
import datetime
import pytz
import numpy as np

Read CSVs in as dataframes using pandas.

In [ ]:
dfPhraseLoad = pd.read_csv('/content/drive/MyDrive/Div Audit 11-10-22/phraseALL_11-10-22.csv', usecols=['record_num', 'is_permuted', 'index_entry'], dtype={'record_num':int, 'is_permuted': str, 'index_entry': str})
dfBibLoad = pd.read_csv('/content/drive/MyDrive/Div Audit 11-10-22/Bibs_11-10-22.csv', usecols=['record_num', 'material_type_name'], dtype={'record_num': int, 'material_type_name': str})
dfItemLoad = pd.read_csv('/content/drive/MyDrive/Div Audit 11-10-22/Items_11-10-22.csv', usecols=['record_num', 'creation_date_gmt', 'call_number_norm', 'location_code', 'price', 'checkout_total'], dtype={'record_num': int, 'creation_date_gmt': str, 'call_number_norm': str, 'location_code': str, 'price': float, 'checkout_total': int})
dfLinkLoad = pd.read_csv('/content/drive/MyDrive/Div Audit 11-10-22/RecordLink_11-10-22.csv', usecols=['bib_record_num', 'item_record_num'], dtype={'bib_record_num': int, 'item_record_num': int})

dfPhraseFull keeps the bib record numbers so that it can merge with the bib report later. dfPhrase pares down the report so that lookups are faster.

In [ ]:
dfPhraseFull = dfPhraseLoad.loc[dfPhraseLoad['is_permuted'].isnull()].reset_index(drop=True)
dfPhraseFull = dfPhraseFull.drop(['is_permuted'], axis=1)

dfPhrase = dfPhraseFull.drop(['record_num'], axis=1)
dfPhrase = dfPhrase.drop_duplicates(subset=['index_entry'])

Create regex patterns and compile them.

In [ ]:
rel_buddPat = r'(\bzen\b)|(dalai lama)|(buddhis)'
rel_hinduPat = r'(\bhindu(?!(stan|\skush)))|(divali)|(\bholi\b)|(bhagavadgita)|(upanishads)'
rel_agnosPat = r'(agnosticism)|(atheism)|(secularism)'
rel_chrisPat = r'(shaker)|(new testament)|(protestant)|(bibl(e|ical))|(nativity)|(adventis)|(mormon)|(baptist)|(catholic)|(methodis)|(pentecost)|(episcopal)|(lutheran)|(clergy)|(church)|(evangelicalism)|(christianity)|(easter\b)|(christmas)|(noahs ark)|(christian(?!.*\d{4}))'
rel_islamPat = r'((?<!terrorism.*)(islam(?!.*(fundamentalism|terrorism))))|(\bsufi(sm)?)|(ramadan)|(id al (fitr\b)|(\badha\b))|(quran)|(sunnites)|(shiah)|(muslim)|(mosques)|(qawwali)'
rel_paganPat = r'(paganism)|(neopagans)|(wicca)'
rel_jewPat = r'(jews)|(jewish)|(judaism)|(holocaust)|(hanukkah)|(purim)|(passover)|(zionis)|(hasid)|(antisemitism)|(rosh hashanah)|(yom kippur)|(sabbath(?!day))|(sukkot)|(pentateuch)|(synagogue)|(yiddish)|(hebrew)'
rel_genPat = r'(\breligio)'

classPat = r'(working class)|(social ((status)|(mobility)|(class)|(stratification)))|(standard of living)|(poor)|(\bcaste\b)|(classism)'

nat_sasiaPat = r'(south asia)|(indic\b)|(\bindia\b)|(east indian)|(bengali)|(bangladesh)|(?<!everest.*)(nepal(?!.*everest))|(sri lanka)|(bhutan)'
nat_easiaPat = r'(east asia)|(asian americans)|(?<!everest).*(chin(a(?!\sfictitious)|ese)(?!.*everest))|(japan)|(korea)|(taiwan)|(vietnam(?! war))|(cambodia)|(mongolia)|(lao(s|tian))|(myanmar)|(malay)|(thai)|(philippin)|(filipino)|(indonesia)|(polynesia)|(brunei)|(east timor)|(pacific island)|(tibet autonomous)|(hmong)'
nat_indigPat = r'(indigenous)|(aboriginal)|(american indian)|((?<!east(ern)\s)\bindians(?!\sbaseball))|(apache)|(cherokee)|(navajo)|(trail of tears)|(aztecs)|(indian art)|(maya(s|n))|(ojibwa)|(iroquois)|(nez perce)|(shoshoni)|(pueblo indian)|(seminole)|(eskimos)|(inuit)|(inca(s|n))|(algonquia?n)|(arctic peoples)|(aleut)'
nat_arabPat = r'(\barab)|(afghan(?!\swar))|(?<!k2.*)(pakistan(?!.*k2))|(middle east)|(palestin)|(bedouin)|(israel)|(saudi)|(yemen)|(iraq(?!\swar))|(\biran\b)|(egypt(?!ologists))|(leban(on|ese))|(qatar)|(syria)|((?<!wild )turk((ish|ey(?!(s| hunting)))?)\b)|(kurdis)|(bahrain)|(cyprus)|(kuwait)|(\boman)|(?<!(belfort|lacey|romero|peele|kisner|lebowitz|miller|myles|reid|rubin|schnitzer|shakoor|sonnenblick|spieth|john|davis|clara|richard) )jordan(?! (ruth|fisher|vernon|michael|barbara|robbie|carol|john|david|grace|family|schnitzer|hal|louis|karl|raisa|dorothy|clarence|bruce|billy|andrew|b\b|wong|will|ted|steve|robert|pete|pat|mattie|marsh|leslie|june|joseph|hamilton|zach|teresa|bella|eben))'
nat_hispPat = r'(hispanic)|(?<!new\s)(mexic)|(latin america)|(cuba(?!n\smissile))|(puerto ric)|(dominican)|(el salvador)|(salvadoran)|(argentin)|(bolivia)|(chile)|(colombia)|(costa rica)|(ecuador)|(equatorial guinea)|(guatemala)|(hondura)|(nicaragua)|(panama)|(paragua)|(peru)|(spain)|(spaniard)|(spanish)|(urugua)|(venezuela)|(brazil)|(guiana)|(guadaloup)|(martinique)|(saint barthelemy)|(saint martin)'
nat_blackPat = r'(men black)|(\bafro)|(haiti)|(blacks(?!mith))|(africa)|(black (nationalism|panther party|power|muslim|lives))|(harlem renaissance)|(abolition)|(segregation)|(?<!(rome)|(italy)|(egypt)).*(slave(s|(ry))(?!(rome)|(egypt)|(italy)))|(slave trade)|(emancipation)|(underground railroad)|(apartheid)|(jamaica)|(nigeria)|(ethiopia)|(congo)|((?<!kilmanjaro.*)(tanzania(?!.*kilmanjaro)))|(kenya)|(uganda)|(sudan)|(ghana)|(cameroon)|(madagascar)|(mozambique)|(angola)|(niger)|(ivory coast)|(\bmali\b)|(burkina faso)|(malawi)|(somalia)|(zambia)|(senegal)|(zimbabw)|(rwanda)|(eritrea)|(guinea (?!pig))|(benin\b)|(burundi)|(sierra leone)|(\btogo\b)|(liberia)|(mauritania)|(\bgabon)|(namibia)|(botswana)|(lesotho)|(gambia)|(eswatini)|(djibouti)|(\btutsi\b)|((?<!(johnson|foster|gardenier|gibbs|hurley|jenkins|kerley|kister|rje) )\bchad\b)'
nat_multiPat = r'(multicultural)|(interracial)|(cross cultural)|(diasporas)|((?<!sexual\s)minorities)|(ethnic identity)|((race|ethnic) relations)|(racially mixed)|(bilingual)|(passing identity)' 

dis_blindPat = r'(blind)'
dis_deafPat = r'(deaf)'
dis_ampPat = r'(amputees)'
dis_otherPat = r'((?<!recordings for people.*)disabilit)|(terminally ill)|(patients)'

men_autismPat = r'(aspergers)|(autis(m|tic))'
men_anxPat = r'(anxiety)'
men_ocdPat = r'(compulsive)'
men_schizoPat = r'(schizophrenia)'
men_eatingPat = r'(eating disorders)'
men_moodPat = r'(suicid)|(depressi(?!ons))|(stress (psychology|disorder))|(postpartum psychiatric disorder)|(seasonal affective disorder)'
men_otherPat = r'(neurobehavioral)|(neuropsychology)|(neurodiversity)|(brain variation)|(personality disorder)|(mentally ill)|(acceptance)|(mental (health|illness|healing))|(resilience personality)|(self (esteem|confidence|realization|perception|actualization|management|destructive|control))|(emotional problems)|(mindfulness)|(psychic trauma)|((?<!(homo|islamo|trans|xeno))phobia)'

add_gamblePat = r'(gamblers)'
add_drugPat = r'(drug use)|(drug abuse)|((substance|medication|opioid|oxycodone|cocaine|marijuana|opium|phetamine|drug|morphine|heroin)\sabuse)'
add_alcoPat = r'(alcoholi(?!c beverages))|(binge drinking)'
add_otherPat = r'((?<!relationship\s)addiction)|(addicts)'

vio_genPat = r'(harassment)|(victims of)|(bullying)|(aggressiveness)|(violent crimes)|((?<!non)violence)|(crimes against)'
vio_hatePat = r'(hate crime)|(internment)'
vio_police = r'(police brutality)'
vio_traffickPat = r'((human|child)\strafficking)|(kidnapping)'
vio_murderPat = r'(genocide)|((?<!(su)|(herb)|(pest))icide)|(suicide bombers)|(murder)'
vio_torturePat = r'((?<!psychological\s)torture)'
vio_rapePat = r'(\brape)'
vio_abusePat = r'((?<!(substance|medication|opioid|oxycodone|cocaine|marijuana|opium|phetamine|drug|morphine|heroin))\sabuse)'

equ_phobiaPat = r'(((islamo)|(xeno)|(trans))phobia)'
equ_racerelPat = r'(racial profiling)|(ku klux klan)|(eugenics)|(race awareness)|(equality)|(racism)|(colorism)'
equ_immigrantPat = r'(immigra)|(refugee)'
equ_genderPat = r'(feminis)|(womens rights)|(sexism)|(suffrag)|(sex role)|(abortion)|(sexual harassment)'
equ_climatePat = r'(sustainable development)|(environmental)|(climatic changes)'
equ_otherPat = r'(persecution)|(activis)|(social psychology)|(social status)|(political prisoners)|((?<!fugitives from )justice(?!(s of the peace)|(\s(league|society|donald))))|(social (change)|(movements)|(problems)|(reformers)|(responsibilit)|(conditions))|(discrimination)|(poverty)|(((pro choice)|(labor)|(gay liberation)|(anti nazi)|(black lives matter)) movement)|((human|civil) rights)|(prejudice)|(protest movements)|(homeless)|(public (health|welfare))'

lgbt_gayPat = r'(gay(s|\b(?!(head|john))))|(homosexual)|(lesbian)|(stonewall riots)'
lgbt_bisexPat = r'(bisexual)'
lgbt_asexPat = r'(asexual)'
lgbt_interPat = r'(intersex)'
lgbt_transPat = r'(trans(gen|sex|phobia))'
lgbt_otherPat = r'(sexual minorities)|(gender)|(masculinity)|(femininity)|(drag show)|(male impersonator)|(queer)|(lgbtq)'

In [ ]:
BUDcomp = re.compile(rel_buddPat)
HINcomp = re.compile(rel_hinduPat)
AGNcomp = re.compile(rel_agnosPat)
CHRcomp = re.compile(rel_chrisPat)
ISLcomp = re.compile(rel_islamPat)
PAGcomp = re.compile(rel_paganPat)
JEWcomp = re.compile(rel_jewPat)
RELcomp = re.compile(rel_genPat)

CLAcomp = re.compile(classPat)

SAScomp = re.compile(nat_sasiaPat)
EAScomp = re.compile(nat_easiaPat)
INDcomp = re.compile(nat_indigPat)
HIScomp = re.compile(nat_hispPat)
BLAcomp = re.compile(nat_blackPat)
ARAcomp = re.compile(nat_arabPat)
MULcomp = re.compile(nat_multiPat)

BLIcomp = re.compile(dis_blindPat)
DEAcomp = re.compile(dis_deafPat)
AMPcomp = re.compile(dis_ampPat)
DIScomp = re.compile(dis_otherPat)

AUTcomp = re.compile(men_autismPat)
ANXcomp = re.compile(men_anxPat)
OCDcomp = re.compile(men_ocdPat)
SCHcomp = re.compile(men_schizoPat)
EATcomp = re.compile(men_eatingPat)
DEPcomp = re.compile(men_moodPat)
MENcomp = re.compile(men_otherPat)

GAMcomp = re.compile(add_gamblePat)
DRUcomp = re.compile(add_drugPat)
ALCcomp = re.compile(add_alcoPat)
ADDcomp = re.compile(add_otherPat)

VIOcomp = re.compile(vio_genPat)
HATcomp = re.compile(vio_hatePat)
POLcomp = re.compile(vio_police)
TRAcomp = re.compile(vio_traffickPat)
MURcomp = re.compile(vio_murderPat)
TORcomp = re.compile(vio_torturePat)
RAPcomp = re.compile(vio_rapePat)
ABUcomp = re.compile(vio_abusePat)

PHOcomp = re.compile(equ_phobiaPat)
SOCcomp = re.compile(equ_racerelPat)
IMMcomp = re.compile(equ_immigrantPat)
GENcomp = re.compile(equ_genderPat)
CLIcomp = re.compile(equ_climatePat)
EQUcomp = re.compile(equ_otherPat)

GAYcomp = re.compile(lgbt_gayPat)
BIScomp = re.compile(lgbt_bisexPat)
ASEcomp = re.compile(lgbt_asexPat)
INTcomp = re.compile(lgbt_interPat)
TNScomp = re.compile(lgbt_transPat)
LGBcomp = re.compile(lgbt_otherPat)

Initialize lists for categories.

In [ ]:
BUDlist = []
HINlist = []
AGNlist = []
CHRlist = []
ISLlist = []
PAGlist = []
JEWlist = []
RELlist = []

CLAlist = []

SASlist = []
EASlist = []
INDlist = []
HISlist = []
BLAlist = []
ARAlist = []
MULlist = []

BLIlist = []
DEAlist = []
AMPlist = []
DISlist = []

AUTlist = []
ANXlist = []
OCDlist = []
SCHlist = []
EATlist = []
DEPlist = []
MENlist = []

GAMlist = []
DRUlist = []
ALClist = []
ADDlist = []

VIOlist = []
HATlist = []
POLlist = []
TRAlist = []
MURlist = []
TORlist = []
RAPlist = []
ABUlist = []

PHOlist = []
SOClist = []
IMMlist = []
GENlist = []
CLIlist = []
EQUlist = []

GAYlist = []
BISlist = []
ASElist = []
INTlist = []
TNSlist = []
LGBlist = []

Create columns with boolean values for whether that category shows up in the subject heading.

In [ ]:
BUDlist = [int(bool(BUDcomp.search(x))) for x in dfPhrase['index_entry']]
HINlist = [int(bool(HINcomp.search(x))) for x in dfPhrase['index_entry']]
AGNlist = [int(bool(AGNcomp.search(x))) for x in dfPhrase['index_entry']]
CHRlist = [int(bool(CHRcomp.search(x))) for x in dfPhrase['index_entry']]
ISLlist = [int(bool(ISLcomp.search(x))) for x in dfPhrase['index_entry']]
PAGlist = [int(bool(PAGcomp.search(x))) for x in dfPhrase['index_entry']]
JEWlist = [int(bool(JEWcomp.search(x))) for x in dfPhrase['index_entry']]
RELlist = [int(bool(RELcomp.search(x))) for x in dfPhrase['index_entry']]

CLAlist = [int(bool(CLAcomp.search(x))) for x in dfPhrase['index_entry']]

SASlist = [int(bool(SAScomp.search(x))) for x in dfPhrase['index_entry']]
EASlist = [int(bool(EAScomp.search(x))) for x in dfPhrase['index_entry']]
INDlist = [int(bool(INDcomp.search(x))) for x in dfPhrase['index_entry']]
HISlist = [int(bool(HIScomp.search(x))) for x in dfPhrase['index_entry']]
ARAlist = [int(bool(ARAcomp.search(x))) for x in dfPhrase['index_entry']]
BLAlist = [int(bool(BLAcomp.search(x))) for x in dfPhrase['index_entry']]
MULlist = [int(bool(MULcomp.search(x))) for x in dfPhrase['index_entry']]

BLIlist = [int(bool(BLIcomp.search(x))) for x in dfPhrase['index_entry']]
DEAlist = [int(bool(DEAcomp.search(x))) for x in dfPhrase['index_entry']]
AMPlist = [int(bool(AMPcomp.search(x))) for x in dfPhrase['index_entry']]
DISlist = [int(bool(DIScomp.search(x))) for x in dfPhrase['index_entry']]

AUTlist = [int(bool(AUTcomp.search(x))) for x in dfPhrase['index_entry']]
ANXlist = [int(bool(ANXcomp.search(x))) for x in dfPhrase['index_entry']]
OCDlist = [int(bool(OCDcomp.search(x))) for x in dfPhrase['index_entry']]
SCHlist = [int(bool(SCHcomp.search(x))) for x in dfPhrase['index_entry']]
EATlist = [int(bool(EATcomp.search(x))) for x in dfPhrase['index_entry']]
DEPlist = [int(bool(DEPcomp.search(x))) for x in dfPhrase['index_entry']]
MENlist = [int(bool(MENcomp.search(x))) for x in dfPhrase['index_entry']]

GAMlist = [int(bool(GAMcomp.search(x))) for x in dfPhrase['index_entry']]
DRUlist = [int(bool(DRUcomp.search(x))) for x in dfPhrase['index_entry']]
ALClist = [int(bool(ALCcomp.search(x))) for x in dfPhrase['index_entry']]
ADDlist = [int(bool(ADDcomp.search(x))) for x in dfPhrase['index_entry']]

VIOlist = [int(bool(VIOcomp.search(x))) for x in dfPhrase['index_entry']]
HATlist = [int(bool(HATcomp.search(x))) for x in dfPhrase['index_entry']]
POLlist = [int(bool(POLcomp.search(x))) for x in dfPhrase['index_entry']]
TRAlist = [int(bool(TRAcomp.search(x))) for x in dfPhrase['index_entry']]
MURlist = [int(bool(MURcomp.search(x))) for x in dfPhrase['index_entry']]
TORlist = [int(bool(TORcomp.search(x))) for x in dfPhrase['index_entry']]
RAPlist = [int(bool(RAPcomp.search(x))) for x in dfPhrase['index_entry']]
ABUlist = [int(bool(ABUcomp.search(x))) for x in dfPhrase['index_entry']]

PHOlist = [int(bool(PHOcomp.search(x))) for x in dfPhrase['index_entry']]
SOClist = [int(bool(SOCcomp.search(x))) for x in dfPhrase['index_entry']]
IMMlist = [int(bool(IMMcomp.search(x))) for x in dfPhrase['index_entry']]
GENlist = [int(bool(GENcomp.search(x))) for x in dfPhrase['index_entry']]
CLIlist = [int(bool(CLIcomp.search(x))) for x in dfPhrase['index_entry']]
EQUlist = [int(bool(EQUcomp.search(x))) for x in dfPhrase['index_entry']]

GAYlist = [int(bool(GAYcomp.search(x))) for x in dfPhrase['index_entry']]
BISlist = [int(bool(BIScomp.search(x))) for x in dfPhrase['index_entry']]
ASElist = [int(bool(ASEcomp.search(x))) for x in dfPhrase['index_entry']]
INTlist = [int(bool(INTcomp.search(x))) for x in dfPhrase['index_entry']]
TNSlist = [int(bool(TNScomp.search(x))) for x in dfPhrase['index_entry']]
LGBlist = [int(bool(LGBcomp.search(x))) for x in dfPhrase['index_entry']]

Convert lists into dataframe columns for dfPhrase.

In [ ]:
dfPhrase['Religion_Buddhism'] = BUDlist
dfPhrase['Religion_Hinduism'] = HINlist
dfPhrase['Religion_Atheism'] = AGNlist
dfPhrase['Religion_Christianity'] = CHRlist
dfPhrase['Religion_Islam'] = ISLlist
dfPhrase['Religion_Paganism'] = PAGlist
dfPhrase['Religion_Judaism'] = JEWlist
dfPhrase['Religion_General'] = RELlist

dfPhrase['Class'] = CLAlist

dfPhrase['Culture_SouthAsia'] = SASlist
dfPhrase['Culture_EastAsia'] = EASlist
dfPhrase['Culture_Indigenous'] = INDlist
dfPhrase['Culture_Hispanic'] = HISlist
dfPhrase['Culture_Black'] = BLAlist
dfPhrase['Culture_Arab'] = ARAlist
dfPhrase['Culture_Multicultural'] = MULlist

dfPhrase['Disability_Blind'] = BLIlist
dfPhrase['Disability_Deaf'] = DEAlist
dfPhrase['Disability_Amputee'] = AMPlist
dfPhrase['Disability_General'] = DISlist

dfPhrase['Mental_Autism'] = AUTlist
dfPhrase['Mental_Anxiety'] = ANXlist
dfPhrase['Mental_Compulsive'] = OCDlist
dfPhrase['Mental_Schizophrenia'] = SCHlist
dfPhrase['Mental_Eating'] = EATlist
dfPhrase['Mental_Depression'] = DEPlist
dfPhrase['Mental_General'] = MENlist

dfPhrase['Addiction_Gambling'] = GAMlist
dfPhrase['Addiction_Drugs'] = DRUlist
dfPhrase['Addiction_Alcohol'] = ALClist
dfPhrase['Addiction_General'] = ADDlist

dfPhrase['Violence_General'] = VIOlist
dfPhrase['Violence_HateCrimes'] = HATlist
dfPhrase['Violence_PoliceBrutality'] = POLlist
dfPhrase['Violence_Trafficking'] = TRAlist
dfPhrase['Violence_Murder'] = MURlist
dfPhrase['Violence_Torture'] = TORlist
dfPhrase['Violence_Rape'] = RAPlist
dfPhrase['Violence_Abuse'] = ABUlist

dfPhrase['Equity_Phobias'] = PHOlist
dfPhrase['Equity_Social'] = SOClist
dfPhrase['Equity_Immigration'] = IMMlist
dfPhrase['Equity_Gender'] = GENlist
dfPhrase['Equity_Climate'] = CLIlist
dfPhrase['Equity_General'] = EQUlist

dfPhrase['LGBT_Gay'] = GAYlist
dfPhrase['LGBT_Bisexual'] = BISlist
dfPhrase['LGBT_Asexual'] = ASElist
dfPhrase['LGBT_Intersex'] = INTlist
dfPhrase['LGBT_Trans'] = TNSlist
dfPhrase['LGBT_General'] = LGBlist

Export boolean dataframe to csv to check whether regex is working.

In [ ]:
dfPhrase.to_csv('drive/My Drive/Div Audit 11-10-22/phraseBools_11-10-22.csv', index=False)

Set up bib record doc: combine and filter material types.

In [ ]:
dfBib = dfBibLoad.replace({'material_type_name':{'BLU-RAY':'Movie', 'DVD OR VCD':'Movie', 'JUV READALONG':'Book', 'BOOK':'Book', 'LARGE PRINT':'Book', 'PLAYAWAY AUDIOBOOK':'Audiobook', 'SPOKEN CD':'Audiobook'}})
dfBib = dfBib.loc[dfBib['material_type_name'].isin(['Book','Movie','Audiobook'])].reset_index(drop=True)

Time to merge stuff. Start by merging the two phrase dataframes (like a vlookup). Then the phrase (subject heading) column can be dropped.

In [ ]:
newPhrase = pd.merge(dfPhraseFull, dfPhrase, on='index_entry')
newPhrase = newPhrase.drop(['index_entry'], axis=1)

Now merge phrase with bib records. Condense the new dataframe using groupby. After this you can try to use the highest count in a column to assign a primary category. But we will move on for now.

In [ ]:
bibSubjects = pd.merge(newPhrase, dfBib, on='record_num')
bibSubjectsCond = bibSubjects.groupby(['record_num','material_type_name']).sum().reset_index()

In [ ]:
religionList = ['Religion_Buddhism', 'Religion_Hinduism', 'Religion_Atheism', 'Religion_Christianity', 'Religion_Islam', 'Religion_Paganism', 'Religion_Judaism', 'Religion_General']
classList = ['Class']
cultureList = ['Culture_SouthAsia', 'Culture_EastAsia', 'Culture_Indigenous', 'Culture_Hispanic', 'Culture_Arab', 'Culture_Black', 'Culture_Multicultural']
disableList = ['Disability_Blind', 'Disability_Deaf', 'Disability_Amputee', 'Disability_General']
mentalList = ['Mental_Autism', 'Mental_Anxiety', 'Mental_Compulsive', 'Mental_Schizophrenia', 'Mental_Eating', 'Mental_Depression', 'Mental_General']
addictList = ['Addiction_Gambling', 'Addiction_Drugs', 'Addiction_Alcohol', 'Addiction_General']
violenceList = ['Violence_General', 'Violence_HateCrimes', 'Violence_PoliceBrutality', 'Violence_Trafficking', 'Violence_Murder', 'Violence_Torture', 'Violence_Rape', 'Violence_Abuse']
equityList = ['Equity_Phobias', 'Equity_Social', 'Equity_Immigration', 'Equity_Gender', 'Equity_Climate', 'Equity_General']
lgbtList = ['LGBT_Gay', 'LGBT_Bisexual', 'LGBT_Asexual', 'LGBT_Intersex', 'LGBT_Trans', 'LGBT_General']

catList = ['Religion_Buddhism', 'Religion_Hinduism', 'Religion_Atheism', 'Religion_Christianity', 'Religion_Islam', 'Religion_Paganism', 'Religion_Judaism', 'Religion_General', 'Class', 'Culture_SouthAsia', 'Culture_EastAsia', 'Culture_Indigenous', 'Culture_Hispanic', 'Culture_Black', 'Culture_Multicultural', 'Disability_Blind', 'Disability_Deaf', 'Disability_Amputee', 'Disability_General', 'Mental_Autism', 'Mental_Anxiety', 'Mental_Compulsive', 'Mental_Schizophrenia', 'Mental_Eating', 'Mental_Depression', 'Mental_General', 'Addiction_Gambling', 'Addiction_Drugs', 'Addiction_Alcohol', 'Addiction_General', 'Violence_General', 'Violence_HateCrimes', 'Violence_PoliceBrutality', 'Violence_Trafficking', 'Violence_Murder', 'Violence_Torture', 'Violence_Rape', 'Violence_Abuse', 'Equity_Phobias', 'Equity_Social', 'Equity_Immigration', 'Equity_Gender', 'Equity_Climate', 'Equity_General', 'LGBT_Gay', 'LGBT_Bisexual', 'LGBT_Asexual', 'LGBT_Intersex', 'LGBT_Trans', 'LGBT_General']

In [ ]:
relArr = bibSubjectsCond[religionList].to_numpy()
sumRelArr = np.sum(relArr, axis=1).tolist()

relOutput = []
relOutput = ['Unique Religion' if sumRelArr[x] > 0 else 'Not Religion' for x in range(len(sumRelArr))]

bibSubjectsCond['Unique Religion'] = relOutput

In [ ]:
classArr = bibSubjectsCond[classList].to_numpy()
sumClaArr = np.sum(classArr, axis=1).tolist()

claOutput = []
claOutput = ['Unique Class' if sumClaArr[x] > 0 else 'Not Class' for x in range(len(sumClaArr))]

bibSubjectsCond['Unique Class'] = claOutput

In [ ]:
cultArr = bibSubjectsCond[cultureList].to_numpy()
sumCultArr = np.sum(cultArr, axis=1).tolist()

cultOutput = []
cultOutput = ['Unique Culture' if sumCultArr[x] > 0 else 'Not Culture' for x in range(len(sumCultArr))]

bibSubjectsCond['Unique Culture'] = cultOutput

In [ ]:
disArr = bibSubjectsCond[disableList].to_numpy()
sumDisArr = np.sum(disArr, axis=1).tolist()

disOutput = []
disOutput = ['Unique Disability' if sumDisArr[x] > 0 else 'Not Disability' for x in range(len(sumDisArr))]

bibSubjectsCond['Unique Disability'] = disOutput

In [ ]:
menArr = bibSubjectsCond[mentalList].to_numpy()
sumMenArr = np.sum(menArr, axis=1).tolist()

menOutput = []
menOutput = ['Unique Mental' if sumMenArr[x] > 0 else 'Not Mental' for x in range(len(sumMenArr))]

bibSubjectsCond['Unique Mental'] = menOutput

In [ ]:
addArr = bibSubjectsCond[addictList].to_numpy()
sumAddArr = np.sum(addArr, axis=1).tolist()

addOutput = []
addOutput = ['Unique Addiction' if sumAddArr[x] > 0 else 'Not Addiction' for x in range(len(sumAddArr))]

bibSubjectsCond['Unique Addiction'] = addOutput

In [ ]:
vioArr = bibSubjectsCond[violenceList].to_numpy()
sumVioArr = np.sum(vioArr, axis=1).tolist()

vioOutput = []
vioOutput = ['Unique Violence' if sumVioArr[x] > 0 else 'Not Violence' for x in range(len(sumVioArr))]

bibSubjectsCond['Unique Violence'] = vioOutput

In [ ]:
equArr = bibSubjectsCond[equityList].to_numpy()
sumEquArr = np.sum(equArr, axis=1).tolist()

equOutput = []
equOutput = ['Unique Equity' if sumEquArr[x] > 0 else 'Not Equity' for x in range(len(sumEquArr))]

bibSubjectsCond['Unique Equity'] = equOutput

In [ ]:
lgbtArr = bibSubjectsCond[lgbtList].to_numpy()
sumLGBTArr = np.sum(lgbtArr, axis=1).tolist()

lgbtOutput = []
lgbtOutput = ['Unique LGBT' if sumLGBTArr[x] > 0 else 'Not LGBT' for x in range(len(sumLGBTArr))]

bibSubjectsCond['Unique LGBT'] = lgbtOutput

In [ ]:
catArr = bibSubjectsCond[catList].to_numpy()
sumCatArr = np.sum(catArr, axis=1).tolist()

divList = []
divList = ['Unique Diverse' if sumCatArr[x] > 0 else 'Not Diverse' for x in range(len(sumCatArr))]

bibSubjectsCond['Unique Diversity'] = divList

Merge Bib/Subject dataframe with record link, in order to link it to the item dataframe later.

In [ ]:
recLink = pd.merge(bibSubjectsCond, dfLinkLoad, left_on='record_num', right_on='bib_record_num').reset_index(drop=True)
recLink = recLink.drop(['record_num'], axis=1)

Merge this dataframe with the item df.

In [ ]:
newItem = pd.merge(dfItemLoad, recLink, left_on='record_num', right_on='item_record_num').reset_index(drop=True)
newItem = newItem.drop(['record_num'], axis=1)

Assign Fiction or NF genre.

In [ ]:
nfPat = r'((.*\d\d\d.*)|(.*poetry.*)|(^jb.*))'
NFcomp = re.compile(nfPat)

GenreList = []
GenreList = ['Nonfiction' if NFcomp.search(x) else 'Fiction' for x in newItem['call_number_norm'].astype(str)]
newItem['Genre'] = GenreList

Get branch location and audience from location code. (Some location codes are entered wrong, so filter out those with only three characters first). Then merge into the final dataframe and drop unnecessary columns.

In [ ]:
newItem = newItem[newItem['location_code'].apply (lambda x: len(str(x)) > 3)]

locs = newItem.filter(['location_code'], axis=1)
locs = locs.drop_duplicates(subset=['location_code'])

AudList = []
AudList = ['Juv' if x[3] == 'j' else 'YA' if x[3] == 'y' else 'Adult' for x in locs['location_code']]

LocList = []
LocList = ['Main' if x[2] == 'm' else 'Outreach' if x[2] == '3' else 'Boudreau' if x[2] == '4' else 'CSQ' if x[2] == '5' else 'Collins' if x[2] == '6' else "O'Connell" if x[2] == '7' else "O'Neill" if x[2] == '8' else 'Valente' if x[2] == '9' else 'Other' for x in locs['location_code']]

locs['Audience'] = AudList
locs['Location'] = LocList

finalItem = pd.merge(newItem, locs, on='location_code')
finalItem = finalItem.drop(['call_number_norm', 'location_code'], axis=1)

Convert creation date to datetime object and get age of item.

In [ ]:
finalItem['creation_date_gmt'] = pd.to_datetime(finalItem['creation_date_gmt'], utc=True)
today = datetime.datetime.now(tz=pytz.utc)

AgeList = []
AgeList = [(today - x).days for x in finalItem['creation_date_gmt']]
AgeListYears = [x/365 for x in AgeList]
finalItem['Years Old'] = AgeListYears

finalItem = finalItem.drop(['creation_date_gmt'], axis=1)

Get averages for age, price, and total number of checkouts.

In [ ]:
dfBud = finalItem.loc[(finalItem['Religion_Buddhism'] > 0) & (finalItem['price'] > 0)]
dfBudAvg = dfBud.groupby(['Location', 'Audience', 'Genre', 'material_type_name'])['price', 'checkout_total', 'Years Old'].mean().reset_index()
dfBudAvg['Category'] = 'Religion_Buddhism' 

dfHin = finalItem.loc[(finalItem['Religion_Hinduism'] > 0) & (finalItem['price'] > 0)]
dfHinAvg = dfHin.groupby(['Location', 'Audience', 'Genre', 'material_type_name'])['price', 'checkout_total', 'Years Old'].mean().reset_index()
dfHinAvg['Category'] = 'Religion_Hinduism'

dfAth = finalItem.loc[(finalItem['Religion_Atheism'] > 0) & (finalItem['price'] > 0)]
dfAthAvg = dfAth.groupby(['Location', 'Audience', 'Genre', 'material_type_name'])['price', 'checkout_total', 'Years Old'].mean().reset_index()
dfAthAvg['Category'] = 'Religion_Atheism'

dfIsl = finalItem.loc[(finalItem['Religion_Islam'] > 0) & (finalItem['price'] > 0)]
dfIslAvg = dfIsl.groupby(['Location', 'Audience', 'Genre', 'material_type_name'])['price', 'checkout_total', 'Years Old'].mean().reset_index()
dfIslAvg['Category'] = 'Religion_Islam'

dfChr = finalItem.loc[(finalItem['Religion_Christianity'] > 0) & (finalItem['price'] > 0)]
dfChrAvg = dfChr.groupby(['Location', 'Audience', 'Genre', 'material_type_name'])['price', 'checkout_total', 'Years Old'].mean().reset_index()
dfChrAvg['Category'] = 'Religion_Christianity'

dfPag = finalItem.loc[(finalItem['Religion_Paganism'] > 0) & (finalItem['price'] > 0)]
dfPagAvg = dfPag.groupby(['Location', 'Audience', 'Genre', 'material_type_name'])['price', 'checkout_total', 'Years Old'].mean().reset_index()
dfPagAvg['Category'] = 'Religion_Paganism'

dfJud = finalItem.loc[(finalItem['Religion_Judaism'] > 0) & (finalItem['price'] > 0)]
dfJudAvg = dfJud.groupby(['Location', 'Audience', 'Genre', 'material_type_name'])['price', 'checkout_total', 'Years Old'].mean().reset_index()
dfJudAvg['Category'] = 'Religion_Judaism'

dfRel = finalItem.loc[(finalItem['Religion_General'] > 0) & (finalItem['price'] > 0)]
dfRelAvg = dfRel.groupby(['Location', 'Audience', 'Genre', 'material_type_name'])['price', 'checkout_total', 'Years Old'].mean().reset_index()
dfRelAvg['Category'] = 'Religion_General'

dfCla = finalItem.loc[(finalItem['Class'] > 0) & (finalItem['price'] > 0)]
dfClaAvg = dfCla.groupby(['Location', 'Audience', 'Genre', 'material_type_name'])['price', 'checkout_total', 'Years Old'].mean().reset_index()
dfClaAvg['Category'] = 'Class'

dfSA = finalItem.loc[(finalItem['Culture_SouthAsia'] > 0) & (finalItem['price'] > 0)]
dfSAAvg = dfSA.groupby(['Location', 'Audience', 'Genre', 'material_type_name'])['price', 'checkout_total', 'Years Old'].mean().reset_index()
dfSAAvg['Category'] = 'Culture_SouthAsia'

dfEA = finalItem.loc[(finalItem['Culture_EastAsia'] > 0) & (finalItem['price'] > 0)]
dfEAAvg = dfEA.groupby(['Location', 'Audience', 'Genre', 'material_type_name'])['price', 'checkout_total', 'Years Old'].mean().reset_index()
dfEAAvg['Category'] = 'Culture_EastAsia'

dfInd = finalItem.loc[(finalItem['Culture_Indigenous'] > 0) & (finalItem['price'] > 0)]
dfIndAvg = dfInd.groupby(['Location', 'Audience', 'Genre', 'material_type_name'])['price', 'checkout_total', 'Years Old'].mean().reset_index()
dfIndAvg['Category'] = 'Culture_Indigenous'

dfHis = finalItem.loc[(finalItem['Culture_Hispanic'] > 0) & (finalItem['price'] > 0)]
dfHisAvg = dfHis.groupby(['Location', 'Audience', 'Genre', 'material_type_name'])['price', 'checkout_total', 'Years Old'].mean().reset_index()
dfHisAvg['Category'] = 'Culture_Hispanic'

dfArab = finalItem.loc[(finalItem['Culture_Arab'] > 0) & (finalItem['price'] > 0)]
dfArabAvg = dfArab.groupby(['Location', 'Audience', 'Genre', 'material_type_name'])['price', 'checkout_total', 'Years Old'].mean().reset_index()
dfArabAvg['Category'] = 'Culture_Arab'

dfBla = finalItem.loc[(finalItem['Culture_Black'] > 0) & (finalItem['price'] > 0)]
dfBlaAvg = dfBla.groupby(['Location', 'Audience', 'Genre', 'material_type_name'])['price', 'checkout_total', 'Years Old'].mean().reset_index()
dfBlaAvg['Category'] = 'Culture_Black'

dfMul = finalItem.loc[(finalItem['Culture_Multicultural'] > 0) & (finalItem['price'] > 0)]
dfMulAvg = dfMul.groupby(['Location', 'Audience', 'Genre', 'material_type_name'])['price', 'checkout_total', 'Years Old'].mean().reset_index()
dfMulAvg['Category'] = 'Culture_Multicultural'

dfBli = finalItem.loc[(finalItem['Disability_Blind'] > 0) & (finalItem['price'] > 0)]
dfBliAvg = dfBli.groupby(['Location', 'Audience', 'Genre', 'material_type_name'])['price', 'checkout_total', 'Years Old'].mean().reset_index()
dfBliAvg['Category'] = 'Disability_Blind'

dfDea = finalItem.loc[(finalItem['Disability_Deaf'] > 0) & (finalItem['price'] > 0)]
dfDeaAvg = dfDea.groupby(['Location', 'Audience', 'Genre', 'material_type_name'])['price', 'checkout_total', 'Years Old'].mean().reset_index()
dfDeaAvg['Category'] = 'Disability_Deaf'

dfAmp = finalItem.loc[(finalItem['Disability_Amputee'] > 0) & (finalItem['price'] > 0)]
dfAmpAvg = dfAmp.groupby(['Location', 'Audience', 'Genre', 'material_type_name'])['price', 'checkout_total', 'Years Old'].mean().reset_index()
dfAmpAvg['Category'] = 'Disability_Amputee'

dfDis = finalItem.loc[(finalItem['Disability_General'] > 0) & (finalItem['price'] > 0)]
dfDisAvg = dfDis.groupby(['Location', 'Audience', 'Genre', 'material_type_name'])['price', 'checkout_total', 'Years Old'].mean().reset_index()
dfDisAvg['Category'] = 'Disability_General'

dfAut = finalItem.loc[(finalItem['Mental_Autism'] > 0) & (finalItem['price'] > 0)]
dfAutAvg = dfAut.groupby(['Location', 'Audience', 'Genre', 'material_type_name'])['price', 'checkout_total', 'Years Old'].mean().reset_index()
dfAutAvg['Category'] = 'Mental_Autism'

dfAnx = finalItem.loc[(finalItem['Mental_Anxiety'] > 0) & (finalItem['price'] > 0)]
dfAnxAvg = dfAnx.groupby(['Location', 'Audience', 'Genre', 'material_type_name'])['price', 'checkout_total', 'Years Old'].mean().reset_index()
dfAnxAvg['Category'] = 'Mental_Anxiety'

dfOCD = finalItem.loc[(finalItem['Mental_Compulsive'] > 0) & (finalItem['price'] > 0)]
dfOCDAvg = dfOCD.groupby(['Location', 'Audience', 'Genre', 'material_type_name'])['price', 'checkout_total', 'Years Old'].mean().reset_index()
dfOCDAvg['Category'] = 'Mental_Compulsive'

dfSch = finalItem.loc[(finalItem['Mental_Schizophrenia'] > 0) & (finalItem['price'] > 0)]
dfSchAvg = dfSch.groupby(['Location', 'Audience', 'Genre', 'material_type_name'])['price', 'checkout_total', 'Years Old'].mean().reset_index()
dfSchAvg['Category'] = 'Mental_Schizophrenia'

dfEat = finalItem.loc[(finalItem['Mental_Eating'] > 0) & (finalItem['price'] > 0)]
dfEatAvg = dfEat.groupby(['Location', 'Audience', 'Genre', 'material_type_name'])['price', 'checkout_total', 'Years Old'].mean().reset_index()
dfEatAvg['Category'] = 'Mental_Eating'

dfDep = finalItem.loc[(finalItem['Mental_Depression'] > 0) & (finalItem['price'] > 0)]
dfDepAvg = dfDep.groupby(['Location', 'Audience', 'Genre', 'material_type_name'])['price', 'checkout_total', 'Years Old'].mean().reset_index()
dfDepAvg['Category'] = 'Mental_Depression'

dfMen = finalItem.loc[(finalItem['Mental_General'] > 0) & (finalItem['price'] > 0)]
dfMenAvg = dfMen.groupby(['Location', 'Audience', 'Genre', 'material_type_name'])['price', 'checkout_total', 'Years Old'].mean().reset_index()
dfMenAvg['Category'] = 'Mental_General'

dfGam = finalItem.loc[(finalItem['Addiction_Gambling'] > 0) & (finalItem['price'] > 0)]
dfGamAvg = dfGam.groupby(['Location', 'Audience', 'Genre', 'material_type_name'])['price', 'checkout_total', 'Years Old'].mean().reset_index()
dfGamAvg['Category'] = 'Addiction_Gambling'

dfDru = finalItem.loc[(finalItem['Addiction_Drugs'] > 0) & (finalItem['price'] > 0)]
dfDruAvg = dfDru.groupby(['Location', 'Audience', 'Genre', 'material_type_name'])['price', 'checkout_total', 'Years Old'].mean().reset_index()
dfDruAvg['Category'] = 'Addiction_Drugs'

dfAlc = finalItem.loc[(finalItem['Addiction_Alcohol'] > 0) & (finalItem['price'] > 0)]
dfAlcAvg = dfAlc.groupby(['Location', 'Audience', 'Genre', 'material_type_name'])['price', 'checkout_total', 'Years Old'].mean().reset_index()
dfAlcAvg['Category'] = 'Addiction_Alcohol'

dfAdd = finalItem.loc[(finalItem['Addiction_General'] > 0) & (finalItem['price'] > 0)]
dfAddAvg = dfAdd.groupby(['Location', 'Audience', 'Genre', 'material_type_name'])['price', 'checkout_total', 'Years Old'].mean().reset_index()
dfAddAvg['Category'] = 'Addiction_General'

dfVio = finalItem.loc[(finalItem['Violence_General'] > 0) & (finalItem['price'] > 0)]
dfVioAvg = dfVio.groupby(['Location', 'Audience', 'Genre', 'material_type_name'])['price', 'checkout_total', 'Years Old'].mean().reset_index()
dfVioAvg['Category'] = 'Violence_General'

dfHat = finalItem.loc[(finalItem['Violence_HateCrimes'] > 0) & (finalItem['price'] > 0)]
dfHatAvg = dfHat.groupby(['Location', 'Audience', 'Genre', 'material_type_name'])['price', 'checkout_total', 'Years Old'].mean().reset_index()
dfHatAvg['Category'] = 'Violence_HateCrimes'

dfPol = finalItem.loc[(finalItem['Violence_PoliceBrutality'] > 0) & (finalItem['price'] > 0)]
dfPolAvg = dfPol.groupby(['Location', 'Audience', 'Genre', 'material_type_name'])['price', 'checkout_total', 'Years Old'].mean().reset_index()
dfPolAvg['Category'] = 'Violence_PoliceBrutality'

dfTra = finalItem.loc[(finalItem['Violence_Trafficking'] > 0) & (finalItem['price'] > 0)]
dfTraAvg = dfTra.groupby(['Location', 'Audience', 'Genre', 'material_type_name'])['price', 'checkout_total', 'Years Old'].mean().reset_index()
dfTraAvg['Category'] = 'Violence_Trafficking'

dfMur = finalItem.loc[(finalItem['Violence_Murder'] > 0) & (finalItem['price'] > 0)]
dfMurAvg = dfMur.groupby(['Location', 'Audience', 'Genre', 'material_type_name'])['price', 'checkout_total', 'Years Old'].mean().reset_index()
dfMurAvg['Category'] = 'Violence_Murder'

dfTor = finalItem.loc[(finalItem['Violence_Torture'] > 0) & (finalItem['price'] > 0)]
dfTorAvg = dfTor.groupby(['Location', 'Audience', 'Genre', 'material_type_name'])['price', 'checkout_total', 'Years Old'].mean().reset_index()
dfTorAvg['Category'] = 'Violence_Torture'

dfRap = finalItem.loc[(finalItem['Violence_Rape'] > 0) & (finalItem['price'] > 0)]
dfRapAvg = dfRap.groupby(['Location', 'Audience', 'Genre', 'material_type_name'])['price', 'checkout_total', 'Years Old'].mean().reset_index()
dfRapAvg['Category'] = 'Violence_Rape'

dfAbu = finalItem.loc[(finalItem['Violence_Abuse'] > 0) & (finalItem['price'] > 0)]
dfAbuAvg = dfAbu.groupby(['Location', 'Audience', 'Genre', 'material_type_name'])['price', 'checkout_total', 'Years Old'].mean().reset_index()
dfAbuAvg['Category'] = 'Violence_Abuse'

dfPho = finalItem.loc[(finalItem['Equity_Phobias'] > 0) & (finalItem['price'] > 0)]
dfPhoAvg = dfPho.groupby(['Location', 'Audience', 'Genre', 'material_type_name'])['price', 'checkout_total', 'Years Old'].mean().reset_index()
dfPhoAvg['Category'] = 'Equity_Phobias'

dfSoc = finalItem.loc[(finalItem['Equity_Social'] > 0) & (finalItem['price'] > 0)]
dfSocAvg = dfSoc.groupby(['Location', 'Audience', 'Genre', 'material_type_name'])['price', 'checkout_total', 'Years Old'].mean().reset_index()
dfSocAvg['Category'] = 'Equity_Social'

dfImm = finalItem.loc[(finalItem['Equity_Immigration'] > 0) & (finalItem['price'] > 0)]
dfImmAvg = dfImm.groupby(['Location', 'Audience', 'Genre', 'material_type_name'])['price', 'checkout_total', 'Years Old'].mean().reset_index()
dfImmAvg['Category'] = 'Equity_Immigration'

dfGen = finalItem.loc[(finalItem['Equity_Gender'] > 0) & (finalItem['price'] > 0)]
dfGenAvg = dfGen.groupby(['Location', 'Audience', 'Genre', 'material_type_name'])['price', 'checkout_total', 'Years Old'].mean().reset_index()
dfGenAvg['Category'] = 'Equity_Gender'

dfCli = finalItem.loc[(finalItem['Equity_Climate'] > 0) & (finalItem['price'] > 0)]
dfCliAvg = dfCli.groupby(['Location', 'Audience', 'Genre', 'material_type_name'])['price', 'checkout_total', 'Years Old'].mean().reset_index()
dfCliAvg['Category'] = 'Equity_Climate'

dfEqu = finalItem.loc[(finalItem['Equity_General'] > 0) & (finalItem['price'] > 0)]
dfEquAvg = dfEqu.groupby(['Location', 'Audience', 'Genre', 'material_type_name'])['price', 'checkout_total', 'Years Old'].mean().reset_index()
dfEquAvg['Category'] = 'Equity_General'

dfGay = finalItem.loc[(finalItem['LGBT_Gay'] > 0) & (finalItem['price'] > 0)]
dfGayAvg = dfGay.groupby(['Location', 'Audience', 'Genre', 'material_type_name'])['price', 'checkout_total', 'Years Old'].mean().reset_index()
dfGayAvg['Category'] = 'LGBT_Gay'

dfBis = finalItem.loc[(finalItem['LGBT_Bisexual'] > 0) & (finalItem['price'] > 0)]
dfBisAvg = dfBis.groupby(['Location', 'Audience', 'Genre', 'material_type_name'])['price', 'checkout_total', 'Years Old'].mean().reset_index()
dfBisAvg['Category'] = 'LGBT_Bisexual'

dfAse = finalItem.loc[(finalItem['LGBT_Asexual'] > 0) & (finalItem['price'] > 0)]
dfAseAvg = dfAse.groupby(['Location', 'Audience', 'Genre', 'material_type_name'])['price', 'checkout_total', 'Years Old'].mean().reset_index()
dfAseAvg['Category'] = 'LGBT_Asexual'

dfInt = finalItem.loc[(finalItem['LGBT_Intersex'] > 0) & (finalItem['price'] > 0)]
dfIntAvg = dfInt.groupby(['Location', 'Audience', 'Genre', 'material_type_name'])['price', 'checkout_total', 'Years Old'].mean().reset_index()
dfIntAvg['Category'] = 'LGBT_Intersex'

dfTrans = finalItem.loc[(finalItem['LGBT_Trans'] > 0) & (finalItem['price'] > 0)]
dfTransAvg = dfTrans.groupby(['Location', 'Audience', 'Genre', 'material_type_name'])['price', 'checkout_total', 'Years Old'].mean().reset_index()
dfTransAvg['Category'] = 'LGBT_Trans'

dfLGB = finalItem.loc[(finalItem['LGBT_General'] > 0) & (finalItem['price'] > 0)]
dfLGBAvg = dfLGB.groupby(['Location', 'Audience', 'Genre', 'material_type_name'])['price', 'checkout_total', 'Years Old'].mean().reset_index()
dfLGBAvg['Category'] = 'LGBT_General'

dfRel1 = finalItem.loc[(finalItem['Unique Religion'] == 'Unique Religion') & (finalItem['price'] > 0)]
dfRel1Avg = dfRel1.groupby(['Location', 'Audience', 'Genre', 'material_type_name'])['price', 'checkout_total', 'Years Old'].mean().reset_index()
dfRel1Avg['Category'] = 'Unique Religion'

dfRel2 = finalItem.loc[(finalItem['Unique Religion'] == 'Not Religion') & (finalItem['price'] > 0)]
dfRel2Avg = dfRel2.groupby(['Location', 'Audience', 'Genre', 'material_type_name'])['price', 'checkout_total', 'Years Old'].mean().reset_index()
dfRel2Avg['Category'] = 'Not Religion'

dfClass1 = finalItem.loc[(finalItem['Unique Class'] == 'Unique Class') & (finalItem['price'] > 0)]
dfClass1Avg = dfClass1.groupby(['Location', 'Audience', 'Genre', 'material_type_name'])['price', 'checkout_total', 'Years Old'].mean().reset_index()
dfClass1Avg['Category'] = 'Unique Class'

dfClass2 = finalItem.loc[(finalItem['Unique Class'] == 'Not Class') & (finalItem['price'] > 0)]
dfClass2Avg = dfClass2.groupby(['Location', 'Audience', 'Genre', 'material_type_name'])['price', 'checkout_total', 'Years Old'].mean().reset_index()
dfClass2Avg['Category'] = 'Not Class'

dfCult1 = finalItem.loc[(finalItem['Unique Culture'] == 'Unique Culture') & (finalItem['price'] > 0)]
dfCult1Avg = dfCult1.groupby(['Location', 'Audience', 'Genre', 'material_type_name'])['price', 'checkout_total', 'Years Old'].mean().reset_index()
dfCult1Avg['Category'] = 'Unique Culture'

dfCult2 = finalItem.loc[(finalItem['Unique Culture'] == 'Not Culture') & (finalItem['price'] > 0)]
dfCult2Avg = dfCult2.groupby(['Location', 'Audience', 'Genre', 'material_type_name'])['price', 'checkout_total', 'Years Old'].mean().reset_index()
dfCult2Avg['Category'] = 'Not Culture'

dfDis1 = finalItem.loc[(finalItem['Unique Disability'] == 'Unique Disability') & (finalItem['price'] > 0)]
dfDis1Avg = dfDis1.groupby(['Location', 'Audience', 'Genre', 'material_type_name'])['price', 'checkout_total', 'Years Old'].mean().reset_index()
dfDis1Avg['Category'] = 'Unique Disability'

dfDis2 = finalItem.loc[(finalItem['Unique Disability'] == 'Not Disability') & (finalItem['price'] > 0)]
dfDis2Avg = dfDis2.groupby(['Location', 'Audience', 'Genre', 'material_type_name'])['price', 'checkout_total', 'Years Old'].mean().reset_index()
dfDis2Avg['Category'] = 'Not Disability'

dfMen1 = finalItem.loc[(finalItem['Unique Mental'] == 'Unique Mental') & (finalItem['price'] > 0)]
dfMen1Avg = dfMen1.groupby(['Location', 'Audience', 'Genre', 'material_type_name'])['price', 'checkout_total', 'Years Old'].mean().reset_index()
dfMen1Avg['Category'] = 'Unique Mental'

dfMen2 = finalItem.loc[(finalItem['Unique Mental'] == 'Not Mental') & (finalItem['price'] > 0)]
dfMen2Avg = dfMen2.groupby(['Location', 'Audience', 'Genre', 'material_type_name'])['price', 'checkout_total', 'Years Old'].mean().reset_index()
dfMen2Avg['Category'] = 'Not Mental'

dfAdd1 = finalItem.loc[(finalItem['Unique Addiction'] == 'Unique Addiction') & (finalItem['price'] > 0)]
dfAdd1Avg = dfAdd1.groupby(['Location', 'Audience', 'Genre', 'material_type_name'])['price', 'checkout_total', 'Years Old'].mean().reset_index()
dfAdd1Avg['Category'] = 'Unique Addiction'

dfAdd2 = finalItem.loc[(finalItem['Unique Addiction'] == 'Not Addiction') & (finalItem['price'] > 0)]
dfAdd2Avg = dfAdd2.groupby(['Location', 'Audience', 'Genre', 'material_type_name'])['price', 'checkout_total', 'Years Old'].mean().reset_index()
dfAdd2Avg['Category'] = 'Not Addiction'

dfVio1 = finalItem.loc[(finalItem['Unique Violence'] == 'Unique Violence') & (finalItem['price'] > 0)]
dfVio1Avg = dfVio1.groupby(['Location', 'Audience', 'Genre', 'material_type_name'])['price', 'checkout_total', 'Years Old'].mean().reset_index()
dfVio1Avg['Category'] = 'Unique Violence'

dfVio2 = finalItem.loc[(finalItem['Unique Violence'] == 'Not Violence') & (finalItem['price'] > 0)]
dfVio2Avg = dfVio2.groupby(['Location', 'Audience', 'Genre', 'material_type_name'])['price', 'checkout_total', 'Years Old'].mean().reset_index()
dfVio2Avg['Category'] = 'Not Violence'

dfEqu1 = finalItem.loc[(finalItem['Unique Equity'] == 'Unique Equity') & (finalItem['price'] > 0)]
dfEqu1Avg = dfEqu1.groupby(['Location', 'Audience', 'Genre', 'material_type_name'])['price', 'checkout_total', 'Years Old'].mean().reset_index()
dfEqu1Avg['Category'] = 'Unique Equity'

dfEqu2 = finalItem.loc[(finalItem['Unique Equity'] == 'Not Equity') & (finalItem['price'] > 0)]
dfEqu2Avg = dfEqu2.groupby(['Location', 'Audience', 'Genre', 'material_type_name'])['price', 'checkout_total', 'Years Old'].mean().reset_index()
dfEqu2Avg['Category'] = 'Not Equity'

dfLGB1 = finalItem.loc[(finalItem['Unique LGBT'] == 'Unique LGBT') & (finalItem['price'] > 0)]
dfLGB1Avg = dfLGB1.groupby(['Location', 'Audience', 'Genre', 'material_type_name'])['price', 'checkout_total', 'Years Old'].mean().reset_index()
dfLGB1Avg['Category'] = 'Unique LGBT'

dfLGB2 = finalItem.loc[(finalItem['Unique LGBT'] == 'Not LGBT') & (finalItem['price'] > 0)]
dfLGB2Avg = dfLGB2.groupby(['Location', 'Audience', 'Genre', 'material_type_name'])['price', 'checkout_total', 'Years Old'].mean().reset_index()
dfLGB2Avg['Category'] = 'Not LGBT'

dfDiv = finalItem.loc[(finalItem['Unique Diversity'] == 'Unique Diverse') & (finalItem['price'] > 0)]
dfDivAvg = dfDiv.groupby(['Location', 'Audience', 'Genre', 'material_type_name'])['price', 'checkout_total', 'Years Old'].mean().reset_index()
dfDivAvg['Category'] = 'Unique Diverse'

dfND = finalItem.loc[(finalItem['Unique Diversity'] == 'Not Diverse') & (finalItem['price'] > 0)]
dfNDAvg = dfND.groupby(['Location', 'Audience', 'Genre', 'material_type_name'])['price', 'checkout_total', 'Years Old'].mean().reset_index()
dfNDAvg['Category'] = 'Not Diverse'

frames = [dfBudAvg, dfHinAvg, dfAthAvg, dfIslAvg, dfChrAvg, dfPagAvg, dfJudAvg, dfRelAvg, dfClaAvg, dfSAAvg, dfEAAvg, dfIndAvg, dfHisAvg, dfArabAvg, dfBlaAvg, dfMulAvg, dfBliAvg, dfDeaAvg, dfAmpAvg, dfDisAvg, dfAutAvg, dfAnxAvg, dfOCDAvg, dfSchAvg, dfEatAvg, dfDepAvg, dfMenAvg, dfGamAvg, dfDruAvg, dfAlcAvg, dfAddAvg, dfVioAvg, dfHatAvg, dfPolAvg, dfTraAvg, dfMurAvg, dfTorAvg, dfRapAvg, dfAbuAvg, dfPhoAvg, dfSocAvg, dfImmAvg, dfGenAvg, dfCliAvg, dfEquAvg, dfGayAvg, dfBisAvg, dfAseAvg, dfIntAvg, dfTransAvg, dfLGBAvg, dfRel1Avg, dfRel2Avg, dfClass1Avg, dfClass2Avg, dfCult1Avg, dfCult2Avg, dfDis1Avg, dfDis2Avg, dfMen1Avg, dfMen2Avg, dfAdd1Avg, dfAdd2Avg, dfVio1Avg, dfVio2Avg, dfEqu1Avg, dfEqu2Avg, dfLGB1Avg, dfLGB2Avg, dfDivAvg, dfNDAvg]

dfAvgs = pd.concat(frames)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a lis

Create dataframe of distinct bib records. Then convert back to boolean values. (The sum function has added up the values for each item, but to count them later on for the bibs, we just want ones and zeroes.)

In [ ]:
dfDisBib = finalItem.groupby(['bib_record_num','Location','Audience','Genre','material_type_name', 'Unique Diversity', 'Unique Religion', 'Unique Class', 'Unique Culture', 'Unique Disability', 'Unique Mental', 'Unique Addiction', 'Unique Violence', 'Unique Equity', 'Unique LGBT'])[catList].sum()

dfDisBib[catList] = dfDisBib[catList].astype(bool)
dfDisBib[catList] = dfDisBib[catList].astype(int)

Count total unique diverse/nondiverse bibs and put them into the Category column.

In [ ]:
dfTotUnique = dfDisBib.groupby(['Location','Audience','Genre','material_type_name', 'Unique Diversity']).size().reset_index(name='Count')
dfTotUnique = dfTotUnique.rename(columns={'Unique Diversity':'Category'})

dfRelUnique = dfDisBib.groupby(['Location','Audience','Genre','material_type_name', 'Unique Religion']).size().reset_index(name='Count')
dfRelUnique = dfRelUnique.rename(columns={'Unique Religion':'Category'})

dfClassUnique = dfDisBib.groupby(['Location','Audience','Genre','material_type_name', 'Unique Class']).size().reset_index(name='Count')
dfClassUnique = dfClassUnique.rename(columns={'Unique Class':'Category'})

dfCultureUnique = dfDisBib.groupby(['Location','Audience','Genre','material_type_name', 'Unique Culture']).size().reset_index(name='Count')
dfCultureUnique = dfCultureUnique.rename(columns={'Unique Culture':'Category'})

dfDisUnique = dfDisBib.groupby(['Location','Audience','Genre','material_type_name', 'Unique Disability']).size().reset_index(name='Count')
dfDisUnique = dfDisUnique.rename(columns={'Unique Disability':'Category'})

dfMentalUnique = dfDisBib.groupby(['Location','Audience','Genre','material_type_name', 'Unique Mental']).size().reset_index(name='Count')
dfMentalUnique = dfMentalUnique.rename(columns={'Unique Mental':'Category'})

dfAddictUnique = dfDisBib.groupby(['Location','Audience','Genre','material_type_name', 'Unique Addiction']).size().reset_index(name='Count')
dfAddictUnique = dfAddictUnique.rename(columns={'Unique Addiction':'Category'})

dfViolenceUnique = dfDisBib.groupby(['Location','Audience','Genre','material_type_name', 'Unique Violence']).size().reset_index(name='Count')
dfViolenceUnique = dfViolenceUnique.rename(columns={'Unique Violence':'Category'})

dfEquityUnique = dfDisBib.groupby(['Location','Audience','Genre','material_type_name', 'Unique Equity']).size().reset_index(name='Count')
dfEquityUnique = dfEquityUnique.rename(columns={'Unique Equity':'Category'})

dfLGBTUnique = dfDisBib.groupby(['Location','Audience','Genre','material_type_name', 'Unique LGBT']).size().reset_index(name='Count')
dfLGBTUnique = dfLGBTUnique.rename(columns={'Unique LGBT':'Category'})

Do count for categories, then transform data to long format from wide. Add in the counts for the diverse/nondiverse bibs we just did. Export to csv.

In [ ]:
dfCount = dfDisBib.groupby(['Location', 'Audience', 'Genre', 'material_type_name'])[catList].sum().reset_index()

dfMelt = pd.melt(dfCount,id_vars=['Location', 'Audience', 'Genre', 'material_type_name'],var_name='Category',value_name='Count')

framesDF = [dfMelt, dfTotUnique, dfRelUnique, dfClassUnique, dfCultureUnique, dfDisUnique, dfMentalUnique, dfAddictUnique, dfViolenceUnique, dfEquityUnique, dfLGBTUnique]
dfAppend = pd.concat(framesDF, ignore_index=True)

dfMerge = pd.merge(dfAppend, dfAvgs, on=['Location', 'Audience', 'Genre', 'material_type_name', 'Category']).reset_index(drop=True)

dfAggs = dfMerge.rename(columns={'material_type_name':'Format', 'price':"Avg Price", 'checkout_total': "Avg Total Checkouts"})


#I don't know why but Class is being duplicated 4 times. Drop duplicates.

dfAggs = dfAggs.drop_duplicates()

dfAggs.to_csv('drive/My Drive/Div Audit 11-10-22/aggregates_11-10-22.csv', index=False)

Unmount Drive to close out.

In [ ]:
drive.flush_and_unmount()